In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
brighton_points = gpd.read_file("../insight_project_dc/brighton/brighton_points.shp")
brighton_sidewalks = gpd.read_file("../insight_project_dc/brighton/brighton_sidewalks/brighton_sidewalks.shp")
brighton_crosswalks = gpd.read_file("../insight_project_dc/brighton/brighton_crosswalks/brighton_crosswalks.shp")

In [3]:
EPSG_4326 = brighton_crosswalks.crs
EPSG_26919 = brighton_sidewalks.crs

In [4]:
brighton_sidewalks = brighton_sidewalks.to_crs(EPSG_4326)

In [ ]:
brighton_sidewalks['SW'] = True
brighton_sidewalks['CW'] = False
brighton_crosswalks['CW'] = True
brighton_crosswalks['SW'] = False
brighton_sidewalks_crosswalks = brighton_sidewalks.append(brighton_crosswalks, sort=False)


In [ ]:
coord_list = list(brighton_sidewalks_crosswalks.geometry.map(lambda x : list(x.coords)))

In [ ]:
from shapely.geometry import Point
nodes = list(map(lambda p : Point(p[0],p[1]), [p for ps in coord_list for p in ps]))

In [ ]:
import networkx as nx
import momepy as my
import osmnx as ox

In [ ]:
brighton = gpd.read_file("/home/adam/brighton.shp")
brighton_poly = brighton.to_crs(EPSG_4326).geometry[0]

In [ ]:
brighton_sw_points = gpd.GeoDataFrame([Point(point) for ls in list(brighton_sidewalks_crosswalks.geometry.map(lambda x : list(x.coords)).values) for point in ls ])

In [ ]:
brighton_sw_points.geometry = brighton_sw_points[0]

In [ ]:
brighton_sw_points['id'] = brighton_sw_points.index

In [ ]:
coord_dict = dict(zip(list(brighton_sw_points.geometry.map(lambda x : tuple(x.coords)[0])), range(len(list(brighton_sw_points.geometry.map(lambda x : x.coords))))))

In [ ]:
from shapely.coords import CoordinateSequence
brighton_sidewalks_crosswalks['u'] = brighton_sidewalks_crosswalks.geometry.map(lambda x : coord_dict[x.coords[0]])
brighton_sidewalks_crosswalks['v'] = brighton_sidewalks_crosswalks.geometry.map(lambda x : coord_dict[x.coords[1]])
brighton_sidewalks_crosswalks['key'] = brighton_sidewalks_crosswalks['SW'].map(lambda x : int(x))

In [ ]:
brighton_sidewalks_crosswalks.crs

In [ ]:
brighton_sw_points['x'] = brighton_sw_points.geometry.map(lambda x : x.coords[0][0])
brighton_sw_points['y'] = brighton_sw_points.geometry.map(lambda x : x.coords[0][1])

In [ ]:
brighton_sw_points.crs = EPSG_4326
brighton_sidewalks_crosswalks.crs = EPSG_4326

In [ ]:
brighton_G = ox.utils_graph.graph_from_gdfs(brighton_sw_points, brighton_sidewalks_crosswalks)
streets_per_node = ox.utils_graph.count_streets_per_node(brighton_G_proj)
null_nodes = [node for node, count in streets_per_node.items() if count == 0]
dead_nodes = [node for node, count in streets_per_node.items() if count == 1]
brighton_G.remove_nodes_from(null_nodes)
len(brighton_G.nodes)

In [ ]:
#brighton_G_u = ox.utils_graph.get_undirected(ox.utils_graph.graph_from_gdfs(brighton_sw_points, brighton_sidewalks_crosswalks))
brighton_G_u = ox.utils_graph.get_undirected(brighton_G)
print(len(brighton_G_u.nodes))
#streets_per_node = ox.utils_graph.count_streets_per_node(brighton_G_u)
#null_nodes = [node for node, count in streets_per_node.items() if count == 0]
#dead_nodes = [node for node, count in streets_per_node.items() if count == 1]

brighton_G_proj =ox.project_graph(brighton_G, to_crs = EPSG_26919)
brighton_G_proj_u =  ox.utils_graph.get_undirected(brighton_G_proj)

In [ ]:
import geocoder
g1 = geocoder.osm("34 Claymoss Rd, Boston, MA")
g3 = geocoder.osm("90 monastery rd, Boston, MA")

In [ ]:
g1

In [ ]:
p1 = ox.projection.project_geometry(Point(g1.json['lng'], g1.json['lat']), to_crs=EPSG_26919)[0].coords[0]
p2 = ox.projection.project_geometry(Point(g3.json['lng'], g3.json['lat']), to_crs=EPSG_26919)[0].coords[0]

In [ ]:
p1 = (g1.json['lng'], g1.json['lat'])
p3 = (g3.json['lng'], g3.json['lat'])
p1, p3

In [ ]:
len(brighton_G_u.nodes)

In [ ]:
n1 = ox.get_nearest_node(brighton_G_proj_u,p1, method='euclidean')
n2 = ox.get_nearest_node(brighton_G_proj_u,p2, method='euclidean')
n3 = ox.get_nearest_node(brighton_G_proj_u,p3, method='euclidean')
n1, n2, n3

In [ ]:
streets_per_node = ox.utils_graph.count_streets_per_node(brighton_G_proj)

In [ ]:
len(brighton_G_proj.nodes)

In [ ]:
nx.shortest_path(brighton_G_proj_u, n1)

In [ ]:
pt = (G.nodes[610]['x'], G.nodes[610]['y'])

In [ ]:
from shapely.ops import nearest_points
G = brighton_G_proj_u
def get_nearest_pt(ID):
    pt = (G.nodes[ID]['x'], G.nodes[ID]['y'])
    near_edge_id = ox.get_nearest_edge(G, pt, return_geom=True)
    return nearest_points(near_edge_id[3], Point(pt))[0]


In [ ]:
streets_per_node = ox.utils_graph.count_streets_per_node(brighton_G_proj)
#null_nodes = [node for node, count in streets_per_node.items() if count == 0]
dead_nodes = [node for node, count in streets_per_node.items() if count == 1]
gdf_edges = ox.utils_graph.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
gdf_edges.columns


In [ ]:
from shapely.ops import split
for node in dead_nodes:
    pt = (G.nodes[node]['y'], G.nodes[node]['x'])
    u,v,key,geom,dist = ox.get_nearest_edge(G, pt, return_dist=True,return_geom=True)
    if dist < 0.3:
        new_geom = split(geom,get_nearest_pt(node))
        split_edge_id = list(gdf_edges[gdf_edges['geometry']==geom].index)[0]
        split_edge = gdf_edges.iloc[split_edge_id]['geometry']
        gdf_edges.iloc[split_edge_id]['geometry'] = new_geom[0]
        new_row = split_edge
        new_row.geometry = new_geom[1]
        gdf_edges.append(gpd.GeoDataFrame(new_row))
    else:
        print(dist)    

In [ ]:
list(brighton_G_proj_u.edges)